In [36]:
import numpy as np 
import pandas as pd

df = pd.read_csv('Downloads\Dsm.csv')

df

,District,Population,Area
0,"Ilala, Dar es Salaam","1,220,611",210.0
1,Kinondoni,"1,775,049",527.0
2,Temeke,"1,368,881",656.0
3,Kigamboni,"162,932",NaN
4,Ubungo,"845,368",260.0


In [37]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="attractions")

In [38]:
df['gcode'] = df['District'].apply(geolocator.geocode) #just to be sure it is Dar es Salaam
df['Latitude'] = [g.latitude for g in df.gcode]
df['Longitude'] = [g.longitude for g in df.gcode]

In [39]:
df

,District,Population,Area,gcode,Latitude,Longitude
0,"Ilala, Dar es Salaam","1,220,611",210.0,"(Ilala, Dar es Salaam, Coastal Zone, Tanzania,...",-6.827253,39.261120
1,Kinondoni,"1,775,049",527.0,"(Kinondoni, Dar es Salaam, Coastal Zone, Tanza...",-6.784067,39.270073
2,Temeke,"1,368,881",656.0,"(Temeke, Dar es Salaam, Coastal Zone, Tanzania...",-6.851162,39.257311
3,Kigamboni,"162,932",NaN,"(Kigamboni, Dar es Salaam, Coastal Zone, 3918,...",-6.825456,39.309409
4,Ubungo,"845,368",260.0,"(Ubungo, Kisiwani, Ubungo, Dar es Salaam, Coas...",-6.793623,39.209661


In [40]:
df.drop("Population", axis=1, inplace=True)
df.drop("Area", axis=1, inplace=True)
df.drop("gcode", axis=1, inplace=True)

In [41]:
df

,District,Latitude,Longitude
0,"Ilala, Dar es Salaam",-6.827253,39.261120
1,Kinondoni,-6.784067,39.270073
2,Temeke,-6.851162,39.257311
3,Kigamboni,-6.825456,39.309409
4,Ubungo,-6.793623,39.209661


In [42]:
import json

import requests
from pandas.io.json import json_normalize

In [43]:
import matplotlib.cm as cm
import matplotlib.colors as colors

In [44]:
from sklearn.cluster import KMeans

In [45]:
import folium

In [69]:
address = 'Dar es Salaam'

geolocator = Nominatim(user_agent="attractions")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Dar es Salaam coordinates: {}, {}.'.format(latitude, longitude))

dsm = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        fill=True,
        fill_color='#00aeef',
        fill_opacity=1,
        parse_html=False).add_to(dsm) 
    
dsm

Dar es Salaam coordinates: -6.8160837, 39.2803583.


In [47]:
CLIENT_ID = '45MMPR3WB0KPVEIB2SO1NLXJIIYWEFND0JLH5XO5T0WME5TC'
CLIENT_SECRET = 'EKAYMMJACFRULXKRHZATHDIQZXTNYMAG3VZ2UW5Y112AX2GM'
VERSION = '20210603'

In [48]:
lat = df.loc[0, 'Latitude']
long = df.loc[0, 'Longitude']

In [49]:
radius = 10000
limit = 210
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    lat, 
    long, 
    radius,
    limit)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=45MMPR3WB0KPVEIB2SO1NLXJIIYWEFND0JLH5XO5T0WME5TC&client_secret=EKAYMMJACFRULXKRHZATHDIQZXTNYMAG3VZ2UW5Y112AX2GM&v=20210603&ll=-6.82725295,39.261120377570634&radius=10000&limit=210'

In [50]:
places = requests.get(url).json()

In [51]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [70]:
venues = places['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-70-1c3ace3321dc>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues)


,name,categories
0,Snocream,Ice Cream Shop
1,Le Grande Restaurant,American Restaurant
2,"Hyatt Regency Dar Es Salaam, The Kilimanjaro",Hotel
3,Samaki Samaki,Seafood Restaurant
4,Princess casino,Casino


In [21]:
print ('{} unique categories in Ilala.'.format(nearby_venues['categories'].value_counts().shape[0]))

32 unique categories in Ilala.


In [53]:
print (nearby_venues['categories'].value_counts()[0:30])

Hotel                   4
Café                    4
Restaurant              4
Bar                     3
African Restaurant      2
Bus Station             2
Airport Terminal        2
Beach                   2
Lounge                  2
Seafood Restaurant      2
Train Station           1
Thai Restaurant         1
Fish & Chips Shop       1
Chinese Restaurant      1
Hotel Bar               1
Ice Cream Shop          1
Market                  1
Ethiopian Restaurant    1
Caribbean Restaurant    1
Sports Bar              1
Casino                  1
Steakhouse              1
American Restaurant     1
Fast Food Restaurant    1
Japanese Restaurant     1
Bakery                  1
BBQ Joint               1
Coffee Shop             1
Pizza Place             1
Pub                     1
Name: categories, dtype: int64


In [54]:
def getNearbyVenues(names, latitudes, longitudes, radius=10000, LIMIT=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']

    
    return(nearby_venues)

In [55]:
dsm_places = getNearbyVenues(names=df['District'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Ilala, Dar es Salaam
Kinondoni
Temeke
Kigamboni
Ubungo


In [84]:
dsm_restaurants = dsm_places[dsm_places['Venue Category'].str.contains('Beach')].reset_index(drop=True)
dsm_restaurants.index = np.arange(1, len(dsm_restaurants)+1)

In [85]:
print (dsm_restaurants['Venue Category'].value_counts())

Beach    10
Name: Venue Category, dtype: int64


In [86]:
top_spots = dsm_restaurants['Venue Category'].value_counts()[0:10].to_frame(name='frequency')
top_spots=top_spots.reset_index()
top_spots.rename(index=str, columns={"index": "Venue_Category", "frequency": "Frequency"}, inplace=True)
top_spots

,Venue_Category,Frequency
0,Beach,10


In [87]:
dsm_restaurants.head(50)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
1,"Ilala, Dar es Salaam",-6.827253,39.261120,Yacht Club,-6.744274,39.275062,Beach
2,"Ilala, Dar es Salaam",-6.827253,39.261120,Mikadi Beach Camp,-6.819225,39.321361,Beach
3,Kinondoni,-6.784067,39.270073,Yacht Club,-6.744274,39.275062,Beach
4,Kinondoni,-6.784067,39.270073,Mikadi Beach Camp,-6.819225,39.321361,Beach
5,Temeke,-6.851162,39.257311,Mikadi Beach Camp,-6.819225,39.321361,Beach
6,Temeke,-6.851162,39.257311,Oysterbay By The Beach,-6.773446,39.282553,Beach
7,Temeke,-6.851162,39.257311,Coco Beach,-6.768204,39.282242,Beach
8,Kigamboni,-6.825456,39.309409,Yacht Club,-6.744274,39.275062,Beach
9,Kigamboni,-6.825456,39.309409,Mikadi Beach Camp,-6.819225,39.321361,Beach
10,Ubungo,-6.793623,39.209661,Yacht Club,-6.744274,39.275062,Beach


In [88]:
districts_restaurant = dsm_restaurants.groupby(['Neighborhood'])['Venue Category'].apply(lambda x: x[x.str.contains('Beach')].count())

In [89]:
districts_restaurant

Neighborhood
Ilala, Dar es Salaam    2
Kigamboni               2
Kinondoni               2
Temeke                  3
Ubungo                  1
Name: Venue Category, dtype: int64

In [90]:
districts_restaurant

Neighborhood
Ilala, Dar es Salaam    2
Kigamboni               2
Kinondoni               2
Temeke                  3
Ubungo                  1
Name: Venue Category, dtype: int64

In [91]:
dsm_restaurants.drop("Neighborhood Latitude", axis=1, inplace=True)
dsm_restaurants.drop("Neighborhood Longitude", axis=1, inplace=True)
dsm_restaurants.drop("Venue Latitude", axis=1, inplace=True)
dsm_restaurants.drop("Venue Longitude", axis=1, inplace=True)

In [92]:
dsm_restaurants

,Neighborhood,Venue,Venue Category
1,"Ilala, Dar es Salaam",Yacht Club,Beach
2,"Ilala, Dar es Salaam",Mikadi Beach Camp,Beach
3,Kinondoni,Yacht Club,Beach
4,Kinondoni,Mikadi Beach Camp,Beach
5,Temeke,Mikadi Beach Camp,Beach
6,Temeke,Oysterbay By The Beach,Beach
7,Temeke,Coco Beach,Beach
8,Kigamboni,Yacht Club,Beach
9,Kigamboni,Mikadi Beach Camp,Beach
10,Ubungo,Yacht Club,Beach


In [83]:
dsm_restaurants.head(50)

,Neighborhood,Venue,Venue Category
1,Temeke,National Museum of Tanzania,Museum
2,Temeke,Makumbusho Village,History Museum
3,Kigamboni,National Museum of Tanzania,Museum
